All datasets of NBA players' per season averages were acquired from [basketball-reference.com](https://www.basketball-reference.com/). For the purpose of demonstrating how the datasets used for obtaining individual NBA player's composite Rank Score for the 2024-2025 season, I have included in the repository the final processed datasets as a comparison. Further instructions will be provided as clarification for the modifications in the datasets, as you go on with this Python notebook.

Datasets acquired from basketball-reference.com were exported as a CSV file in Excel. When exporting, do not add the "Rk" and "Awards" columns. Below are the links of the sources for the datasets:


*   [2019-20 NBA Player Stats: Per Game](https://www.basketball-reference.com/leagues/NBA_2020_per_game.html)
*   [2020-21 NBA Player Stats: Per Game](https://www.basketball-reference.com/leagues/NBA_2021_per_game.html)
*   [2021-22 NBA Player Stats: Per Game](https://www.basketball-reference.com/leagues/NBA_2022_per_game.html)
*   [2022-23 NBA Player Stats: Per Game](https://www.basketball-reference.com/leagues/NBA_2023_per_game.html)
*   [2023-24 NBA Player Stats: Per Game](https://www.basketball-reference.com/leagues/NBA_2024_per_game.html)
*   [2024-25 NBA Player Stats: Per Game](https://www.basketball-reference.com/leagues/NBA_2025_per_game.html)



In [ ]:
from pathlib import Path
from IPython.display import display
import pandas as pd
import numpy as np
import re
from collections import defaultdict
import sys
sys.path.insert(0, str(Path.cwd().resolve().parent))  # add repo root to sys.path

from project_paths import (
    RAW_DIR, EDITED_DIR, FINAL_DIR, ANALYSIS_DIR, TEMP_DIR, DATA_DIR, ROOKIES_PATH
)

INPUT_DIR    = RAW_DIR
GLOB_PATTERN = "*Season.csv"   # e.g., "2021-2022 Season.csv"
OUTPUT_DIR   = TEMP_DIR
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

In [ ]:
# Column Names from basketball-reference.com site
COLUMN_NAMES = [
    'Player', 'Age', 'Team', 'Pos', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%',
    '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'FT', 'FTA', 'FT%', 'ORB',
    'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PTS', 'Player-additional'
]

In [ ]:
# For datasets without first row header
def load_csv_force_schema(path: Path) -> pd.DataFrame:
    """Load a CSV that may or may not have a header row; force expected schema and coerce numerics."""
    # First try: no header
    df = pd.read_csv(path, header=None, dtype=str)
    if df.shape[1] == len(COLUMN_NAMES):
        df.columns = COLUMN_NAMES
    else:
        # Second try: with header
        try:
            df2 = pd.read_csv(path)
            if list(df2.columns[:len(COLUMN_NAMES)]) == COLUMN_NAMES:
                df = df2.copy()
            else:
                # Fallback: trim or pad columns to expected width
                df = df.iloc[:, :len(COLUMN_NAMES)].copy()
                if df.shape[1] < len(COLUMN_NAMES):
                    for _ in range(len(COLUMN_NAMES) - df.shape[1]):
                        df[df.shape[1]] = np.nan
                df.columns = COLUMN_NAMES
        except Exception:
            # Last resort: enforce width from first attempt
            df = df.iloc[:, :len(COLUMN_NAMES)].copy()
            if df.shape[1] < len(COLUMN_NAMES):
                for _ in range(len(COLUMN_NAMES) - df.shape[1]):
                    df[df.shape[1]] = np.nan
            df.columns = COLUMN_NAMES

    # Coerce numeric columns where sensible
    numeric_cols = [
        'Age','G','GS','MP','FG','FGA','3P','3PA','2P','2PA','FT','FTA',
        'ORB','DRB','TRB','AST','STL','BLK','TOV','PTS'
    ]
    for c in numeric_cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors='coerce')
    return df

# Players' Registered Suffixes
_SUFFIXES = {"Jr.", "Sr.", "II", "III", "IV", "V"}

# Changing of viewing names with Lastname first in All Caps.
def reformat_player_name(name: str) -> str:
    """
    Convert 'First [Middles] Last [Suffix]' -> 'LAST, First [Middles] [Suffix]'.
    Preserves middle names; supports suffixes in _SUFFIXES.
    """
    if not isinstance(name, str):
        return name
    s = name.strip()
    if not s:
        return s
    parts = s.split()
    has_suffix = parts[-1] in _SUFFIXES
    if has_suffix and len(parts) >= 3:
        suffix = parts[-1]
        last = parts[-2]
        first = parts[0]
        middle = " ".join(parts[1:-2])
        return f"{last.upper()}, {first}" + (f" {middle}" if middle else "") + f" {suffix}"
    elif len(parts) >= 2:
        last = parts[-1]
        first = parts[0]
        middle = " ".join(parts[1:-1])
        return f"{last.upper()}, {first}" + (f" {middle}" if middle else "")
    else:
        return s

# Pipeline step for Reformating player names from source datasets
def step_reformat_names(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df['Player'] = df['Player'].astype(str).apply(reformat_player_name)
    return df

# Pipeline step for determining if a player changed teams midseason
def step_duplicate_and_reorder_team_pos(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    # Duplicate
    df['NEW'] = df['Team']
    df['Pos2'] = df['Pos']
    # Rename originals
    df = df.rename(columns={'Team': 'OLD', 'Pos': 'Pos1'})
    # Reorder to place NEW after OLD and Pos2 after Pos1
    cols = list(df.columns)
    def move_after(cols_list, col_to_move, anchor):
        cols_list = cols_list.copy()
        if col_to_move in cols_list and anchor in cols_list:
            cols_list.remove(col_to_move)
            idx = cols_list.index(anchor)
            cols_list.insert(idx + 1, col_to_move)
        return cols_list
    cols = move_after(cols, 'NEW', 'OLD')
    cols = move_after(cols, 'Pos2', 'Pos1')
    return df[cols]

# Pipeline step for dropping categories not used in analysis
def step_drop_unwanted(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    columns_to_delete = [
        'FG', 'FGA', '3PA', '3P%', '2P', '2PA', '2P%', 'FT', 'FTA',
        'ORB', 'DRB', 'Player-additional'
    ]
    drop_cols = [c for c in columns_to_delete if c in df.columns]
    return df.drop(columns=drop_cols)

# Pipeline step for input of change of teams for players who transferred teams mid season
def step_apply_old_new_changes(df: pd.DataFrame) -> pd.DataFrame:
    """
    Within each file, for consecutive duplicate Players:
      - Set the FIRST row's 'OLD' to the second-to-last row's OLD
      - Set the FIRST row's 'NEW' to the last row's NEW
    All rows remain; only the first row of each duplicate block is updated.
    """
    df = df.copy()
    if not {'OLD', 'NEW', 'Player'}.issubset(df.columns):
        return df

    i = 0
    n = len(df)
    while i < n:
        player = df.loc[i, 'Player']
        start = i
        # Advance through consecutive duplicates of the same Player
        while i + 1 < n and df.loc[i + 1, 'Player'] == player:
            i += 1

        if i > start:  # found a block of duplicates [start..i]
            # second-to-last OLD -> first row's OLD
            df.at[start, 'OLD'] = df.at[i - 1, 'OLD']
            # last NEW -> first row's NEW
            df.at[start, 'NEW'] = df.at[i, 'NEW']

        i += 1
    return df

In [ ]:
# Define pipeline order:
#    - Names -> duplicate/reorder team/pos -> drop cols -> per-file duplicate logic
BASE_STEPS = [
    step_reformat_names,
    step_duplicate_and_reorder_team_pos,
    step_apply_old_new_changes,
    step_drop_unwanted
]

def run_pipeline_on_file(path: Path, steps=BASE_STEPS) -> pd.DataFrame:
    df = load_csv_force_schema(path)
    for step in steps:
        df = step(df)
    return df

In [ ]:
# Discover files and process (save each season to Excel)
files = sorted(INPUT_DIR.glob(GLOB_PATTERN))
if not files:
    raise FileNotFoundError(f"No files matched {INPUT_DIR}/{GLOB_PATTERN}")

In [ ]:
# Save files to TEMP folder
processed = []

for f in files:
    cleaned = run_pipeline_on_file(f)  # includes per-file duplicate check
    out_name = f"{f.stem}.xlsx"   # e.g., "2019-2020 Season.xlsx"
    out_path = OUTPUT_DIR / out_name
    cleaned.to_excel(out_path, index=False)  # requires openpyxl or xlsxwriter
    processed.append(str(out_path))

display({
    "Processed files": [f.name for f in files],
    "Saved Excel files": processed,
})

# Peek at a processed season
try:
    peek_latest = processed[5]
    preview_df = pd.read_excel(peek_latest)
    display(preview_df.head(10))
except Exception as e:
    print("Preview skipped:", e)

**This next step is important.**

Before we proceed, since there are now two columns for the most played playing positions of players, we need to manually modify them.

A good example is the Luka Dončić trade of 2024-2025. During his [time in Dallas for that season](https://www.basketball-reference.com/teams/DAL/2025.html#all_pbp_stats) (Look at the play-by-play section if link does not directly load there) he played 73% of the time he was on the court as a Point Guard, and 27% as Shooting Guard, while [as a Laker](https://www.basketball-reference.com/teams/LAL/2025.html#all_pbp_stats) he played 45% as a Point Guard, 43% as a Shooting Guard and 12% as a Small Forward. For simplicity in determining his two most dominant playing positions, he must have played at least 60% of the total percentage allowed for each position. So as a Maverick, given that he played two positions (PG and SG), the allowed percentage for each position is an even 50%. So 60% of 50% should be 30% in order for a position to qualify as dominant playing position for him. Since he played only 27% as a Shooting Guard, for his time in Dallas in 2024-2025 he can qualify as only playing the Point Guard position in both columns for playing positions (Pos1 and Pos2).

As a Laker, we can change his entry there as Pos1 = PG, and Pos2 = SG, given that he played 43% or more for both positions. Had he played 20% as a Small Forward and the remaining two positions also had 20% or more, We would list his Pos1 as Point Guard (given that it would be his most dominant position), and his Pos2 = SF. Although he qualified as SG as well, we go in sequence of playing positions (PG -> SG -> SF -> PF -> C or PG <- SG <- SF <- PF <- C) when a player is dominant in multiple positions. This will be further clarified when computing for the per season rank score of players.

If you wish to skip this part, I have included in the repository the edited datasets required for this part of the data cleaning. The folder is titled as "EDITED". I also edited names for 'Tristan da Silva' and 'Luc Mbah a Moute' given that their last names have more than one word.

In [ ]:
# Load the manually edited Excel files
MANUAL_DIR   = EDITED_DIR
MANUAL_DIR.mkdir(parents=True, exist_ok=True)

# Final output folder
FINAL_DIR   = FINAL_DIR
FINAL_DIR.mkdir(parents=True, exist_ok=True)

MANUAL_GLOB = "*Season.xlsx"  # matches names like "2019-2020 Season.xlsx"


In [ ]:
# Function to get proper Pos1 and Pos2 values
def calculate_positions(df):
    # Define the order of positions for adjacency checks
    position_order = ['PG', 'SG', 'SF', 'PF', 'C']

    # Group data by player to handle each player's entries
    grouped = df.groupby('Player')

    # Iterate over each player's group
    for player, group in grouped:
        # Only process players with multiple entries
        if len(group) > 1:
            # Separate the first entry (season-long stats) and team-specific entries
            season_long_entry = group.iloc[0]  # First entry
            team_entries = group.iloc[1:]       # All other entries

            # Initialize a dictionary to store total counts for each position
            position_counts = defaultdict(int)

            # Start from the most recent team entry to the one before the first entry
            for _, row in team_entries[::-1].iterrows():  # Reverse order
                pos1, pos2 = row['Pos1'], row['Pos2']
                games_played = row['G']

                # Add games played to the respective position counts
                position_counts[pos1] += games_played
                if pos1 != pos2:
                    # Check if positions are adjacent or have intermediate positions
                    pos1_index = position_order.index(pos1)
                    pos2_index = position_order.index(pos2)

                    # If adjacent, just count pos2
                    if abs(pos1_index - pos2_index) == 1:
                        position_counts[pos2] += games_played
                    # If they skip 1 or 2 positions, include intermediates
                    elif abs(pos1_index - pos2_index) <= 2:
                        # Determine range based on order in the sequence
                        start, end = sorted([pos1_index, pos2_index])
                        for i in range(start + 1, end):
                            intermediate_position = position_order[i]
                            position_counts[intermediate_position] += games_played
                        # Count pos2 as well
                        position_counts[pos2] += games_played

            # Determine the most and second-most frequent positions based on counts
            sorted_positions = sorted(position_counts.items(), key=lambda x: x[1], reverse=True)

            # Update season-long entry with the calculated Pos1 and Pos2
            if sorted_positions:
                # Assign most frequent position
                season_long_entry['Pos1'] = sorted_positions[0][0]  # Most frequent position

                # Assign second most frequent position or fallback to Pos1
                if len(sorted_positions) > 1:
                    season_long_entry['Pos2'] = sorted_positions[1][0]  # Second most frequent position
                else:
                    season_long_entry['Pos2'] = sorted_positions[0][0]  # Fallback to the same as Pos1 if only one position

            # Update the original DataFrame
            df.loc[season_long_entry.name] = season_long_entry

    return df  # Returns updated DataFrame with calculated Pos1 and Pos2 for each player

# Function to drop non-total entries of players who got traded/switched teams
def drop_duplicates_keep_first(df):
    # Drop duplicates based on the 'Player' column, keeping only the first entry
    df_no_duplicates = df.drop_duplicates(subset=['Player'], keep='first')

    # Reset the index after dropping duplicates for a cleaner output
    df_no_duplicates.reset_index(drop=True, inplace=True)

    return df_no_duplicates

In [ ]:
manual_files = sorted(MANUAL_DIR.glob(MANUAL_GLOB))
if not manual_files:
    raise FileNotFoundError(f"No files matched {MANUAL_DIR}/{MANUAL_GLOB}")

final_paths = []
for x in manual_files:
    df_manual = pd.read_excel(x)

    # Ensure G is numeric for position weighting
    if 'G' in df_manual.columns:
        df_manual['G'] = pd.to_numeric(df_manual['G'], errors='coerce').fillna(0).astype(int)

    # Apply your functions (as-is)
    df_manual = calculate_positions(df_manual)
    df_manual = drop_duplicates_keep_first(df_manual)

    out_path = FINAL_DIR / x.name.replace(".xlsx", "_final.xlsx")
    df_manual.to_excel(out_path, index=False)
    final_paths.append(str(out_path))

display({
    "Manually edited inputs": [f.name for f in manual_files],
    "Saved final Excel files": final_paths,
})

# Peek at a final_paths season
try:
    peek_latest = final_paths[5]
    preview_df = pd.read_excel(peek_latest)
    display(preview_df.head(10))
except Exception as e:
    print("Preview skipped:", e)